# import

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import dwave_networkx as dnx
import networkx as nx
%matplotlib inline
import matplotlib as mpl
import math
from dwave.system import DWaveSampler
import minorminer as mm
import dwave.inspector

In [101]:
f = open("token.txt", "r")
sampler = DWaveSampler(solver=dict(topology__type='pegasus'),token=f.read())
print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system6.1 was selected.


# target graph (pegasus)

In [3]:
#create a target graph
T=nx.Graph()
T.add_edges_from(sampler.edgelist)
T.add_nodes_from(sampler.nodelist)

# one long chain

In [58]:
#create a source graph
S=nx.Graph()
#spin chain length
N=3500
#define the nodes
S.add_nodes_from(list(range(N)))
spins=list(S.nodes)
#define the edges. the first node is also the last node (PBC)
for i in range(len(spins)-1):
    S.add_edge(spins[i],spins[i+1])
#impose PBC
S.add_edge(spins[len(spins)-1],spins[0])

In [59]:
cond=0
count=0

while (cond==0 and count<1000):
    print(count)

    embedding=mm.find_embedding(S,T)

    #here we make sure that minorminor didn't make any chains 
    biggerthan1=[]
    for value in embedding.values():
        if len(value)>1:
            biggerthan1.append(value)
    #if the embedding has any chains we reject it and try again
    if len(biggerthan1)>0:
        print(len(biggerthan1),"chains")
        cond=0
        count+=1
    if count==999:
        print("reached max iterations")   
    
    #if there are no chains it's a goood embedding
    if len(biggerthan1)==0:
        cond=1
    
print(embedding)

0
6 chains
1


KeyboardInterrupt: embedding cancelled by keyboard interrupt

In [32]:
couplers={}
for edge in list(S.edges):
    spin0=edge[0]
    spin1=edge[1]
    qubit0=embedding.get(spin0)[0]
    qubit1=embedding.get(spin1)[0]
    couplers.update({(qubit0,qubit1):-1})

# many small chains

In [97]:
#create a source graph
S=nx.Graph()
#spin chain length
Nlen=10
#number of chains
Nchains=5

#define the nodes in base chain
base_nodes=[]
for i in range(Nlen):
    base_nodes.append(i)
base_nodes=np.array(base_nodes)
#generate all the nodes
allnodes=[]
for i in range(Nchains):
    allnodes.append(list(base_nodes+Nlen*i))
#add nodes to source graph
for i in range(len(allnodes)):
    for j in range(len(allnodes[i])):
        S.add_node(allnodes[i][j])

#define the edges
for i in range(len(allnodes)):
    spinlist=allnodes[i]
    for j in range(len(spinlist)-1):
        S.add_edge(spinlist[j],spinlist[j+1])
    S.add_edge(spinlist[len(spinlist)-1],spinlist[0])

In [99]:
cond=0
count=0

while (cond==0 and count<1000):
    print(count)

    embedding=mm.find_embedding(S,T)

    #here we make sure that minorminor didn't make any chains 
    biggerthan1=[]
    for value in embedding.values():
        if len(value)>1:
            biggerthan1.append(value)
    #if the embedding has any chains we reject it and try again
    if len(biggerthan1)>0:
        print(len(biggerthan1),"chains")
        cond=0
        count+=1
    if count==999:
        print("reached max iterations")   
    
    #if there are no chains it's a goood embedding
    if len(biggerthan1)==0:
        cond=1
    
print(embedding)

0
{0: [4031], 1: [4030], 9: [2105], 2: [1986], 3: [4165], 4: [4166], 5: [2181], 6: [4136], 7: [2166], 8: [4046], 10: [5463], 11: [749], 19: [5462], 12: [5448], 13: [5449], 14: [929], 15: [5599], 16: [5598], 17: [5597], 18: [554], 20: [879], 21: [4669], 29: [878], 22: [1044], 23: [1043], 24: [4535], 25: [1148], 26: [1163], 27: [4505], 28: [4504], 30: [5236], 31: [447], 39: [327], 32: [5206], 33: [387], 34: [5071], 35: [341], 36: [4951], 37: [311], 38: [326], 40: [1457], 41: [1458], 49: [3292], 42: [3592], 43: [3591], 44: [1293], 45: [1308], 46: [1307], 47: [3381], 48: [1366]}


In [100]:
couplers={}
for edge in list(S.edges):
    spin0=edge[0]
    spin1=edge[1]
    qubit0=embedding.get(spin0)[0]
    qubit1=embedding.get(spin1)[0]
    couplers.update({(qubit0,qubit1):-1})

# anneal

In [33]:
response=sampler.sample_ising(h={},J=couplers,annealing_time=1,num_reads=10)
print(response)
dwave.inspector.show(response)

  31 32 33 34 35 36 38 48 49 50 51 53 55 56 58 59 65 ... 5729  energy num_oc.
0 -1 -1 -1 -1 +1 +1 -1 -1 -1 +1 +1 -1 -1 -1 -1 +1 +1 ...   -1 -2780.0       1
1 +1 +1 +1 +1 +1 +1 -1 +1 +1 +1 +1 -1 +1 +1 -1 -1 +1 ...   -1 -2780.0       1
2 -1 -1 -1 -1 -1 -1 +1 -1 -1 -1 -1 +1 +1 +1 +1 +1 +1 ...   -1 -2776.0       1
3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...   -1 -2776.0       1
4 -1 -1 -1 -1 +1 +1 -1 +1 +1 +1 +1 -1 +1 +1 -1 -1 -1 ...   -1 -2772.0       1
5 -1 -1 -1 -1 +1 +1 +1 +1 +1 +1 +1 +1 +1 +1 -1 -1 -1 ...   -1 -2764.0       1
6 -1 -1 -1 -1 +1 +1 -1 +1 +1 +1 +1 -1 -1 -1 +1 +1 +1 ...   -1 -2760.0       1
7 +1 +1 +1 +1 -1 -1 +1 -1 -1 -1 -1 -1 +1 +1 +1 +1 +1 ...   +1 -2756.0       1
8 -1 -1 -1 -1 +1 +1 +1 -1 +1 +1 +1 +1 -1 -1 +1 +1 -1 ...   +1 -2756.0       1
9 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 +1 +1 +1 ...   -1 -2744.0       1
['SPIN', 10 rows, 10 samples, 3000 variables]


'http://127.0.0.1:18000/?problemId=d0bca4ee-7084-49e0-84fc-04efaca6a431'